In [8]:
import pandas as pd
import numpy as np
import datetime
from functools import reduce
import os

# --- Step 1: Load the data ---
file_path = "E8_InBase_North_Toe_SinkHole_DipoleDipole_FNT2410D_trial37_InvRes8.dat"
df = pd.read_csv(file_path, sep='\s+', header=None, names=["X", "Depth", "Resistivity"])

# --- Step 2: Convert depth to positive ---
df["Depth"] = df["Depth"].abs()

# --- Step 3: Define requested X values ---
requested_x_values = [140,225]

# --- Step 4: Determine available X range ---
available_x = df["X"].unique()
x_min, x_max = available_x.min(), available_x.max()

# --- Step 5: Process valid values only ---
pivoted_logs = []
for req_x in requested_x_values:
    if req_x < x_min or req_x > x_max:
        print(f"❌ Requested X = {req_x} is out of bounds ({x_min} to {x_max}). Skipping.")
        continue

    # Find nearest available X
    nearest_x = available_x[np.abs(available_x - req_x).argmin()]
    
    # Extract and format data
    log_df = df[df["X"] == nearest_x][["Depth", "Resistivity"]].copy().reset_index(drop=True)
    depth_col = f"Depth_{nearest_x}"
    resist_col = f"Resistivity_{nearest_x}"
    log_df.columns = [depth_col, resist_col]

    # Add metadata rows
    metadata = pd.DataFrame({
        depth_col: [f"Requested_X: {req_x}", None],
        resist_col: [f"Nearest_X: {nearest_x}", None]
    })

    combined = pd.concat([metadata, log_df], ignore_index=True)
    pivoted_logs.append(combined)

# --- Step 6: Merge logs side by side if any valid data ---
if pivoted_logs:
    merged_df = reduce(lambda left, right: pd.concat([left, right], axis=1), pivoted_logs)

    # --- Step 7: Save to CSV ---
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.splitext(os.path.basename(file_path))[0]
    output_filename = f"{filename}_{timestamp}.csv"
    merged_df.to_csv(output_filename, index=False)
    print(f"File saved: {output_filename}")
else:
    print("No valid requested X values. No file created.")

File saved: E8_InBase_North_Toe_SinkHole_DipoleDipole_FNT2410D_trial37_InvRes8_20250517_125650.csv
